Dowload financial data from yahoo finance

In [2]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

# Define Gold Symbol on Yahoo
gold_symbol = 'GC=F'  # Contrat à terme sur l'or

# Period of data extraction
start_date = "2020-01-01"
end_date = "2023-01-01"

# Download gold data
gold_data = yf.download(gold_symbol, start=start_date, end=end_date)



[*********************100%***********************]  1 of 1 completed


Selection of metrics

In [ ]:
# Ajouter des métriques supplémentaires
gold_data['Daily Returns'] = gold_data['Close'].pct_change()  # Variation journalière
gold_data['Volume'] = gold_data['Volume']  # Volume de trading
gold_data['Moving Average (20 days)'] = gold_data['Close'].rolling(window=20).mean()  # Moyenne mobile sur 20 jours

# Garder uniquement les colonnes nécessaires
gold_data = gold_data[['Close', 'Daily Returns', 'Volume', 'Moving Average (20 days)']]

# Afficher un aperçu des données
print(gold_data.head())



Missing data imputation and download it in a csv file

In [ ]:
# Vérifier les valeurs manquantes
print("\nValeurs manquantes :\n", gold_data.isnull().sum())

# Remplir les valeurs manquantes (si nécessaire)
gold_data.fillna(method='ffill', inplace=True)

# Enregistrer les données dans un fichier CSV
gold_data.to_csv('gold_data_metrics.csv')

Visualize our data

In [ ]:
# Visualiser les données
plt.figure(figsize=(10, 6))
plt.plot(gold_data['Close'], label='Gold Price')
plt.plot(gold_data['Moving Average (20 days)'], label='20-day Moving Average')
plt.title('Prix de l\'or avec Moyenne Mobile')
plt.xlabel('Date')
plt.ylabel('Prix ($)')
plt.legend()
plt.show()

# Visualisation des rendements journaliers
plt.figure(figsize=(10, 6))
plt.plot(gold_data['Daily Returns'], label='Daily Returns', alpha=0.7)
plt.title('Variation journalière du prix de l\'or')
plt.xlabel('Date')
plt.ylabel('Variation (%)')
plt.legend()
plt.show()

# Visualisation du volume
plt.figure(figsize=(10, 6))
plt.plot(gold_data['Volume'], label='Volume de Trading', alpha=0.7)
plt.title('Volume de trading de l\'or')
plt.xlabel('Date')
plt.ylabel('Volume')
plt.legend()
plt.show()
